## **A1.4 Selección de características**

**1.** Para comenzar se importa la base de datos "*A1.4 Vino Tinto.csv*" con la función `read_csv` de la librería **pandas**. Utilizando la función `.head(5)` se imprimen las primeras 5 filas para observar la estructura de los datos.

In [95]:
import pandas as pd
df = pd.read_csv("A1.4 Vino Tinto.csv")
rows, cols = df.shape
print("Las dimensiones de la base de datos son de",rows,"filas x",cols,"columnas")
df.head(5)

Las dimensiones de la base de datos son de 1599 filas x 12 columnas


,acidezFija,acidezVolatil,acidoCitrico,azucarResidual,cloruros,dioxidoAzufreLibre,dioxidoAzufreTotal,densidad,pH,sulfatos,alcohol,calidad
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


**2.** Para comenzar a trabajar con modelos se **separa la base de datos** en dos con una proporción de** 80% entranamiento** y **20% prueba**. Esto se hace utilizando la función `train_test_split` de la librería `sklearn.model_selection` la cuál divide el dataset en 2 de manera aleatoria para evitar sesgos. Se imprime un resumen de ambos data sets y sus dimensiones para comprobar que mantienen el mismo número de observaciones en conjunto que la base de datos original.

In [96]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
print("Training Set:")
print(train)
print("\nTesting Set:")
print(test)
total = train.shape[0] + test.shape[0]

print(f"Cantidad de observaciones: {total}")

Training Set:
      acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
493          8.7          0.690          0.31             3.0     0.086   
354          6.1          0.210          0.40             1.4     0.066   
342         10.9          0.390          0.47             1.8     0.118   
834          8.8          0.685          0.26             1.6     0.088   
705          8.4          1.035          0.15             6.0     0.073   
...          ...            ...           ...             ...       ...   
1130         9.1          0.600          0.00             1.9     0.058   
1294         8.2          0.635          0.10             2.1     0.073   
860          7.2          0.620          0.06             2.7     0.077   
1459         7.9          0.200          0.35             1.7     0.054   
1126         5.8          0.290          0.26             1.7     0.063   

      dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  \
493        

**3.** En la siguiente celda se utiliza el método de **selección hacia adelante** para seleccionar las características que mejor funcionen para un modelo de **regresión lineal múltiple**. Se preparan los datos de entrenamiento aislando las variables de entrada y la de salida y utilizando la función `Linear Regression` de `sklearn.linear_model` se genera un modelo de regresión lineal el cual se adapte a los datos de entrenamiento.

Para la selección se utiliza la función `SequentialFeatureSelector` de `mlxtend.feature_selection` y se le da de parámetro el modelo generado previamente. Además se establece el parámetro `forward` como `True` para trabajar con selección hacia adelante y se selecciona la **R^2** como métrica para medir el desempeño de las variables. El parámetro `k_features` se escoge como `'best'` con el objetivo de que se seleccione el número de variables que mejor le funcione al modelo para tener un mejor ajuste a los datos.

Una vez hecho el proceso, se imprimen los índices y nombres de las variables que resultaron ser las mejores para el modelo.

In [97]:
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

x_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]

model = LinearRegression()
model.fit(x_train, y_train)
sfs_f = SFS(model, k_features = 'best',forward = True, scoring = 'r2',cv = 10, verbose = 1)
sfs_f.fit(x_train,y_train)
selected_features_f = list(sfs_f.k_feature_names_)
print("\n✅ Características seleccionadas:")
print(f"Índices: {sfs_f.k_feature_idx_}")
print(f"Nombres: {selected_features_f}")


Features: 11/11


✅ Características seleccionadas:
Índices: (1, 4, 5, 6, 8, 9, 10)
Nombres: ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


**4.** Una vez escogidas las variables más relevantes para el modelo, se entrena un nuevo modelo que solamente toma en cuenta estas variables, en este caso 7. El entrenamiento se logra generando vectores nuevos de variables que **solo contengan los datos de las variables previamente seleccionadas** y utilizando la función `.fit` con estos datos. Este nuevo modelo se se prueba con los datos de prueba utilizando la función `.predict` y midiendo el desempeño del modelo con estos datos nuevos. Se importa la función `r2_score` de `sklearn.metrics` para medir el **desempeño del modelo**. Se imprimen las variables usadas en el modelo y su respectivo valor de **R^2 para los datos de prueba**.

In [98]:
from sklearn.metrics import r2_score
x_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]
x_train_selected_f = x_train[selected_features_f]
x_test_selected_f = x_test[selected_features_f]

model.fit(x_train_selected_f, y_train)

y_pred_f = model.predict(x_test_selected_f)
r2_f = r2_score(y_test,y_pred_f)

print("\n✅ Modelo entrenado con las siguientes características seleccionadas:")
print(selected_features_f)
print(f"\n📊 R^2 en datos de prueba: {r2_f:.4f}")


✅ Modelo entrenado con las siguientes características seleccionadas:
['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']

📊 R^2 en datos de prueba: 0.4013


**5.** En la siguiente celda se utiliza el método de **selección hacia atrás** y se repite el proceso de la sección 3 con algunos cambios en el uso de la función `SequentialFeatureSelector`.

El parámetro `forward` como `False` para trabajar con selección hacia atrás y se sigue seleccionando **R^2** como métrica para medir el desempeño de las variables. El parámetro `k_features` se sigue seleccionando como `'best'` con el objetivo de que se seleccione el número de variables que mejor le funcione al modelo para tener un mejor ajuste a los datos.

Una vez hecho el proceso, se imprimen los índices y nombres de las variables que resultaron ser las mejores para el modelo.

In [99]:
sfs_b = SFS(model, k_features = 'best',forward = False, scoring = 'r2',cv = 10, verbose = 1)
sfs_b.fit(x_train,y_train)
selected_features_b = list(sfs_b.k_feature_names_)
print("\n✅ Características seleccionadas:")
print(f"Índices: {sfs_b.k_feature_idx_}")
print(f"Nombres: {selected_features_b}")

Features: 1/1


✅ Características seleccionadas:
Índices: (1, 4, 5, 6, 8, 9, 10)
Nombres: ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


**6.** Una vez escogidas las variables más relevantes para el modelo hacia atrás, se repite la sección 4 y se entrena un nuevo modelo que solamente toma en cuenta las variables seleccionadas.

Se imprimen las variables usadas en el modelo y su respectivo valor de **R^2 para los datos de prueba**.

In [100]:
x_train_selected_b = x_train[selected_features_b]
x_test_selected_b = x_test[selected_features_b]

model.fit(x_train_selected_b, y_train)

y_pred_b = model.predict(x_test_selected_b)
r2_b = r2_score(y_test,y_pred_b)

print("\n✅ Modelo entrenado con las siguientes características seleccionadas:")
print(selected_features_b)
print(f"\n📊 R^2 en datos de prueba: {r2_b:.4f}")

print(f"\nEl modelo de selección hacia adelante tiene un R^2 de {r2_f}")
print(f"y el modelo de selección hacia atrás de {r2_b}, el R^2 de ambos")
print("modelos es igual, lo cual en mi opinión sucede debido a que el dataset es muy pequeño")
print("y al utilizar el parámetro 'best' para encontrar el mejor ajuste en cantidad")
print("de variables, ambos modelos resultan iguales. Por lo cuál en estos casos, ambos tipos")
print("de selección entregan el mismo resultado.")


✅ Modelo entrenado con las siguientes características seleccionadas:
['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']

📊 R^2 en datos de prueba: 0.4013

El modelo de selección hacia adelante tiene un R^2 de 0.4012628835440285
y el modelo de selección hacia atrás de 0.4012628835440285, el R^2 de ambos
modelos es igual, lo cual en mi opinión sucede debido a que el dataset es muy pequeño
y al utilizar el parámetro 'best' para encontrar el mejor ajuste en cantidad
de variables, ambos modelos resultan iguales. Por lo cuál en estos casos, ambos tipos
de selección entregan el mismo resultado.


### **Conclusión**
Tras analizar los resultados obtenidos de ambos métodos de selección se pudo observar que **ambos fueron practicamente idénticos** y se desempeñaron de la misma manera. Esto se debe mayor mente a dos cosas:

* Se utilizó el parámetro `'best'` el cuál buscaba el número de variables que mejor desempeño tuviera para el modelo.

* La base de datos es relativamente
pequeña y tiene **pocas variables y observaciones** que generen diferencias entre ambos tipos de selección.

Para una base de datos como la que se trabajó, ambos métodos de selección son igual de eficientes y entregan los mismos resultados, por lo cuál **ninguno es mejor que el otro**.

*Cabe recalcar que esto no se mantiene para bases de datos más grandes en las cuales el tipo de selección si genera una diferencia notable en el desempeño del modelo y que sí existirían diferencias si el parámetro `k_features` hubiera sido modificado para tener más o menos variables en cada uno de los modelos.*


## Referencias
Cortez, P., Cerdeira, A., Almeida, F., Matos, T., & Reis, J. (2009). Wine Quality [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C56S3T.